In [1]:
import cirq

In [2]:
'''
Takes a qubitString as input and returns a matrix 
that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a bitstring, return the decimal number 
represented by the bitstring
'''
def getDecimalNo(bitString):
    val = 0
    n = len(bitString)
    for i in range(0,n):
        val = val + (2**i)*(int)(bitString[-i-1])
    return val

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

In [3]:
import numpy as np

class FunctionObject:
    def __init__(self,fx,n):
        self.__fx = fx
        self.__n = n
        self.__Uf = self.__createUf()
    
    '''
    Apply the function on the given input.
    '''
    def applyFx(self,input):
        return self.__fx(input)
        
    '''
    The N value which signifies the no. of qubits.
    '''
    def getN(self):
        return self.__n
    
    '''
    The Uf matrix which represents the oracle for this function
    '''
    def getUf(self):
        return self.__Uf
    
    '''
    Given the function to execute, and the qubits |x1x2..xn> and the ancilla|b>.
    This function will return a bit vector representing |x1x2..xn> |b+f(x)>
    '''
    def getFunctionResult(self,x,ancilla):
        fx = self.applyFx(x)
        newAncilla = str((((int)(ancilla) + fx)%2))
        result = x+newAncilla
        return getQubitVector(result)
    
    '''
    Using pointer to f(x) and n denoting the no. of qubits,
    it returns an oracle matrix Uf of size 2^(n+1)x2^(n+1) which
    can be used in a Deutsch-Jozsa or Bernstein-Vazirani circuit
    '''
    def __createUf(self):
        Uf = np.zeros((2**(self.__n+1),2**(self.__n+1)))    
    
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0

        inputs = getAllPossibleNBitStrings(self.__n+1)

        for i in inputs:
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = i[0:self.__n]

            #fx represents the output of function f given the input x
            fx = str(self.applyFx(x))

            #b is the last bit of the bit pattern in the dictionary item
            b = i[self.__n]

            #below we have the (f(x) + b) mod 2
            if(b==fx):
                bfx = '0'
            else:
                bfx = '1'
                # print(bfx)

            #the final bit string is the concatenation of the input x and bfx
            result = x + bfx

            #using indices_dict we can now find the index that corresponds to this output
            column = getDecimalNo(result)
            row = getDecimalNo(i)
            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[row][column] = 1        
        return Uf
    
    '''
    Given a function and n qubits. This function will verify if the Uf matrix 
    generated for this function matches the expected output. 
    i.e., it checks if Uf|x1x2..xn>|b> = |x1x2..xn> |b+f(x)>
    '''
    def verifyUf(self,debug=False):
        
        UfMatrix = self.getUf()
        if debug:
            print("\nUfMatrix for function \n"+str(UfMatrix)+"\n")

        inputs = getAllPossibleNBitStrings(self.__n+1)
        valid = True

        for i in inputs:
            inputBitVector = getQubitVector(i)

            if debug:
                print("Input BitString {}".format(i))
                print("Input BitVector {}\n".format(inputBitVector))

            functionOutput = self.getFunctionResult(i[0:self.__n],i[self.__n])
            UfOutput = np.matmul(UfMatrix,inputBitVector)

            if debug:
                print("\tUf Output {}".format(UfOutput))
                print("\t F Output {}\n".format(functionOutput))

            if np.array_equal(UfOutput,functionOutput) is not True:
                print("ERROR FOR {}\n".format(i))
                valid=False

        return valid

In [4]:
import random

class DeutschJozsaFunction(FunctionObject):
    
    '''
    Initializes a function object with the passed in fx and ftype values.
    If no fx and ftype values are passed, 
    it will create a balanced or constant function at random and initialize it
    '''
    def __init__(self,n, fx=None,ftype=None):
        if fx is None:
            functionObj = self.createRandomDeutschJozsaFunction(n)
            fx = functionObj.fx
            self.ftype = functionObj.getFunctionType()
        else:
            self.ftype = ftype
        FunctionObject.__init__(self, fx, n)        
        
    '''
    Randomly create a balanced or constant function
    '''    
    def createRandomDeutschJozsaFunction(self,n):
        fTypeOrdinal = random.randint(0,1)
        if fTypeOrdinal == 0:
            return BalancedFunction(n)
        else:
            return ConstantFunction(n)
        
    '''
    CONSTANT or BALANCED based on function type 
    '''
    def getType(self):
        return self.ftype
    

class BalancedFunction(DeutschJozsaFunction):
    
    def __init__(self,n):
        # a random bit chosen from 0 to n-1 which will be 
        # 0 for half inputs and 1 for other half inputs
        self.decisionBit = random.randint(0,n-1)
        
    def fx(self,inputString):
        if inputString[self.decisionBit] == '1':
            return 1
        else:
            return 0
    
    def getFunctionType(self):
        return "BALANCED"
    
class ConstantFunction(DeutschJozsaFunction):
    def __init__(self,n):
        # a random int 0 or 1, which will be the output of this constant function
        self.result = random.randint(0,1)
        
    def fx(self,inputString):
        return self.result
    
    def getFunctionType(self):
        return "CONSTANT"

In [5]:
class Oracle(cirq.Gate):
    def __init__(self, n, UfMatrix, name):
        self.__n=n
        self.__UfMatrix=UfMatrix
        self.__name=name
    def num_qubits(self):
        return self.__n 
    def _unitary_(self):
        #return self.__UfMatrix
        #print("Unitary array", np.squeeze(np.asarray(self.__UfMatrix)))
        return np.squeeze(np.asarray(self.__UfMatrix))
        
    def __str__(self):
        return self.__name

In [6]:
decisionBit = 0

def constantFunction(x):
    return 1

def balancedFunction(x):
    if x[decisionBit] == '1':
        return 1
    else:
        return 0

In [7]:
def runMainCircuit(functionObj,debug=False):
    #create a matrix representing Uf
    UfMatrix = functionObj.getUf()
    
    #for a n bit function, we need n+1 qubits(one ancilla bit)
    n = functionObj.getN()+1
    
    # creating an instance of Uf    
    c=cirq.Circuit()
    
    
    qubits = cirq.LineQubit.range(n)
    # setting last qubit to 1
    c.append(cirq.X(qubits[n-1]))
    
    # adding Hadamard gates to all qubits
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])
    
    
    # creating Uf gate
    uf_bv= Oracle(n, UfMatrix, "UF_BV")
    
    # adding Uf gate
    c.append(uf_bv(*qubits))
    
    # helper bit does not require H gate. Result is treated as trash/ garbage    
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])        
    
    # measurements
    for i in range(0,n):
        c.append(cirq.measure(qubits[i])) 
       
    if debug:
        print(c)
    
    simulator = cirq.Simulator()
    
    result = simulator.run(c, repetitions=1)
    
    if debug:
        print(result)
    
    return result

In [8]:
import time

'''
Given a Deutsch-Jozsa functionObject
this function returns if its balanced or constant
'''
def isBalancedOrConstant(functionObj):
    start = time.time()
    results = runMainCircuit(functionObj)
    end = time.time()
    constant = True        
    
    if len(results.measurements) == 0:
        return None,None
    
    for i in range(0,functionObj.getN()):
        if results.measurements[str(i)][0][0] == True:
            constant = False
            break
    
    timetaken = end-start
    
    if constant:
        return "CONSTANT",timetaken
    else:
        return "BALANCED",timetaken

In [9]:
'''
Driver that verifies the validity of the quantum circuits for different F configurations.
Here F is either constant or balanced
'''
class DeutschJozsaDriver():
    def __init__(self,n,fx,fType):
        self.functionObj = DeutschJozsaFunction(n,fx,fType)
        
    def runAndVerifyQuantumCircuit(self,debug=False):
        print("*"*20)
        if debug:
            print("Verified that the created Uf matrix is valid: {}".format(self.functionObj.verifyUf()))

        fType,circuitRunTime = isBalancedOrConstant(self.functionObj)
        if fType is None:
            return
        assert fType == self.functionObj.getType()
        print("N : {} , FnType: {}, Time : {}".format(self.functionObj.getN(), self.functionObj.getType(), circuitRunTime))
        print("*"*20)

In [10]:
decisionBit = 0

def constantFunction(x):
    return 1

def balancedFunction(x):
    if x[decisionBit] == '1':
        return 1
    else:
        return 0

## Simulation 1

In [11]:
DeutschJozsaDriver(1,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()

********************
N : 1 , FnType: CONSTANT, Time : 0.009521961212158203
********************
********************
N : 2 , FnType: CONSTANT, Time : 0.0019259452819824219
********************
********************
N : 3 , FnType: CONSTANT, Time : 0.001989126205444336
********************
********************
N : 4 , FnType: CONSTANT, Time : 0.0055980682373046875
********************
********************
N : 5 , FnType: CONSTANT, Time : 0.005923271179199219
********************
********************
N : 6 , FnType: CONSTANT, Time : 0.015568017959594727
********************


In [12]:
DeutschJozsaDriver(1,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()

********************
N : 1 , FnType: BALANCED, Time : 0.0023119449615478516
********************
********************
N : 2 , FnType: BALANCED, Time : 0.0035719871520996094
********************
********************
N : 3 , FnType: BALANCED, Time : 0.009259939193725586
********************
********************
N : 4 , FnType: BALANCED, Time : 0.004884958267211914
********************
********************
N : 5 , FnType: BALANCED, Time : 0.006680011749267578
********************
********************
N : 6 , FnType: BALANCED, Time : 0.008448123931884766
********************


## Simulation 2

In [13]:
DeutschJozsaDriver(1,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()

********************
N : 1 , FnType: CONSTANT, Time : 0.0031239986419677734
********************
********************
N : 2 , FnType: CONSTANT, Time : 0.0024607181549072266
********************
********************
N : 3 , FnType: CONSTANT, Time : 0.0026650428771972656
********************
********************
N : 4 , FnType: CONSTANT, Time : 0.0046389102935791016
********************
********************
N : 5 , FnType: CONSTANT, Time : 0.006316184997558594
********************
********************
N : 6 , FnType: CONSTANT, Time : 0.008366107940673828
********************


In [14]:
DeutschJozsaDriver(1,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()

********************
N : 1 , FnType: BALANCED, Time : 0.0021810531616210938
********************
********************
N : 2 , FnType: BALANCED, Time : 0.0032057762145996094
********************
********************
N : 3 , FnType: BALANCED, Time : 0.0055561065673828125
********************
********************
N : 4 , FnType: BALANCED, Time : 0.005141019821166992
********************
********************
N : 5 , FnType: BALANCED, Time : 0.006400108337402344
********************
********************
N : 6 , FnType: BALANCED, Time : 0.013624906539916992
********************


## Simulation 3

In [15]:
DeutschJozsaDriver(1,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()

********************
N : 1 , FnType: CONSTANT, Time : 0.0030159950256347656
********************
********************
N : 2 , FnType: CONSTANT, Time : 0.0028791427612304688
********************
********************
N : 3 , FnType: CONSTANT, Time : 0.0070269107818603516
********************
********************
N : 4 , FnType: CONSTANT, Time : 0.00463414192199707
********************
********************
N : 5 , FnType: CONSTANT, Time : 0.008278131484985352
********************
********************
N : 6 , FnType: CONSTANT, Time : 0.00882720947265625
********************


In [16]:
DeutschJozsaDriver(1,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()

********************
N : 1 , FnType: BALANCED, Time : 0.0029671192169189453
********************
********************
N : 2 , FnType: BALANCED, Time : 0.01464986801147461
********************
********************
N : 3 , FnType: BALANCED, Time : 0.007035017013549805
********************
********************
N : 4 , FnType: BALANCED, Time : 0.005400896072387695
********************
********************
N : 5 , FnType: BALANCED, Time : 0.014781951904296875
********************
********************
N : 6 , FnType: BALANCED, Time : 0.011170625686645508
********************
